In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('/home/carlo/workspace/networkqit/')
import networkqit as nq
import numpy as np
import matplotlib.pyplot as plt
import bct

In [ ]:
W=np.loadtxt('/home/carlo/workspace/communityalg/data/GroupAverage_rsfMRI_unthr.adj')
t=0.5
Wt = bct.threshold_absolute(W,t)[0:128,0:128]
t=Wt[np.nonzero(Wt)].min()
plt.imshow(Wt)
plt.colorbar()
print(bct.density_und(Wt),'thresh=',t)

In [ ]:
beta=10
size=[512,512]
W=np.random.exponential(beta,size=size)
W=np.triu(W,1)
W=W + W.T
t=0
Wt=bct.threshold_absolute(W,t)
Wt = np.triu(Wt,1)
Wt += Wt.T
t=Wt[np.nonzero(Wt)].min()
plt.imshow(Wt)
plt.colorbar()
print(bct.density_und(Wt),'thresh=',t)

In [ ]:
M=nq.CWTECM(N=len(Wt),threshold=Wt[np.nonzero(Wt)].min())
sol = M.fit(G=Wt, method='MLE')
L_mle = M.loglikelihood(G=Wt,theta=sol['x'])

In [ ]:
sol = M.fit(G=Wt,method='saddle_point',x0=sol['x'],verbose=2,maxiter=2)
L_saddle = M.loglikelihood(G=Wt, theta=sol['x'])
print(L_mle,L_saddle)

In [ ]:
M.expected_weighted_adjacency(sol['x'])+t

In [ ]:
nq.plot_mle(Wt,M.expected_adjacency(sol['x']), M.expected_weighted_adjacency(sol['x']))
plt.savefig('cwtcm.pdf',bbox='tight',dpi=200)

In [ ]:
X = M.sample_adjacency(sol['x'],batch_size=200)
print('Sample statistics:\tmin=%f\tmax=%f\tmean=%f\tstd=%f' % (X[np.nonzero(X)].min(),X[np.nonzero(X)].max(),X[np.nonzero(X)].mean(),X[np.nonzero(X)].std()))
print('Data statistics:\tmin=%f\tmax=%f\tmean=%f\tstd=%f' % (Wt[np.nonzero(Wt)].min(),Wt[np.nonzero(Wt)].max(),Wt[np.nonzero(Wt)].mean(),Wt[np.nonzero(Wt)].std()))
print('Average diff=%f thresh=%f' % (X[np.nonzero(X)].mean()-Wt[np.nonzero(Wt)].mean(),t))

In [ ]:
np.mean(np.mean(M.sample_adjacency(sol['x'],batch_size=100),0) - Wt.mean(0))

In [ ]:
nq.plot_mle(Wt, np.mean(M.sample_adjacency(sol['x'],batch_size=100)>0,0),
                np.mean(M.sample_adjacency(sol['x'],batch_size=100),0))

In [ ]:
X=M.sample_adjacency(sol['x'],batch_size=100)
_ = plt.hist(X[np.nonzero(X)],label='model',density=True,bins=100,alpha=0.5)
_ = plt.hist(Wt[np.nonzero(Wt)],label='data',density=True,bins=100,alpha=0.5)
plt.legend()

In [ ]:
plt.plot(X[np.nonzero(X)],'.',alpha=0.5,label='model')
plt.plot(Wt[np.nonzero(Wt)],'.',alpha=0.5,label='data')
plt.legend()

In [ ]:
np.mean(X[np.nonzero(X)][0:2000]/Wt[np.nonzero(Wt)][0:2000])

In [ ]:
plt.plot(np.unique(Wt[np.nonzero(Wt)]),label='data')
plt.plot(((np.unique(X[np.nonzero(X)]))),label='model')
plt.legend()

In [ ]:
Y=nq.matrices.trunc_exp_rv(low=10,scale=1,size=[100000,])
_=plt.hist(Y,density=True,bins=200)

In [ ]:
B = np.squeeze(M.expected_weighted_adjacency(sol['x'])) 
S = np.squeeze(M.sample_adjacency(sol['x'],batch_size=1))

def andd_bu(G):
    A = (G>0).astype(float)
    k = A.sum(0)
    return np.einsum('ij,i->j',G,k)/k

def andd_wu(G):
    A = (G>0).astype(float)
    s = G.sum(0)
    k = A.sum(0)
    return np.einsum('ij,i->j',G,s)/k

def clu_bu(G):
    n = G.shape[0]
    d = (1-np.eye(n))
    return np.einsum('ij,jk,ki->i',A*d,A*d,A*d)/np.einsum('ij,ki->i',A*d,A*d)

In [ ]:
#import matplotlib2tikz
plt.plot(bct.clustering_coef_wu(np.squeeze(M.sample_adjacency(sol['x']))),bct.clustering_coef_wu(B),'.r')
#plt.plot(bct.clustering_coef_wu(Wt),bct.clustering_coef_wu(S),'.b')
plt.plot(np.linspace(0,1,10),np.linspace(0,1,10),'-k')
plt.xlabel('k^{nn}')
plt.ylabel('$\\langle k^{nn}\\rangle$')
plt.legend(loc='best')
plt.title('cTWECM')
plt.savefig('cTWECM.pdf',bbox='tight')

# Do the UBCM plots

In [ ]:
M1 = nq.UBCM(N=len(Wt))
sol1 = M1.fit(G=(Wt>0).astype(float), method='MLE')
M1.expected_adjacency(sol1['x'])

In [ ]:
plt.plot(bct.clustering_coef_bu((Wt>0).astype(float)), bct.clustering_coef_wu(M1.expected_adjacency(sol1['x'])),'.',label='Expected')
plt.plot(bct.clustering_coef_bu((Wt>0).astype(float)), bct.clustering_coef_bu(np.squeeze(M1.sample_adjacency(sol1['x']))),'.',label='Sampled')
plt.legend(loc='best')
plt.plot(np.linspace(0,1,10),np.linspace(0,1,10),'-k')

In [ ]:
plt.plot(bct.clustering_coef_bu((Wt>0).astype(float)), bct.clustering_coef_wu(M1.expected_adjacency(sol1['x'])),'.',label='Expected')
plt.plot(bct.clustering_coef_bu((Wt>0).astype(float)), bct.clustering_coef_bu(np.squeeze(M1.sample_adjacency(sol1['x']))),'.',label='Sampled')
plt.legend(loc='best')
plt.plot(np.linspace(0,1,10),np.linspace(0,1,10),'-k')

In [ ]:
plt.plot(andd((Wt>0).astype(float)), andd(M1.expected_adjacency(sol1['x'])),'.',label='Expected')
plt.plot(andd((Wt>0).astype(float)), andd(np.squeeze(M1.sample_adjacency(sol1['x'],batch_size=1))),'.',label='Sampled')
plt.plot(np.linspace(15,50,10),np.linspace(15,50,10),'-k')
plt.xlabel('k^{nn}')
plt.ylabel('$\\langle k^{nn}\\rangle$')
plt.legend(loc='best')
plt.title('Binarized - UBCM')

In [ ]:
def clu(A):
    np.einsum('ijk->l',A)